In [1]:
# pip install xgboost

https://www.kaggle.com/code/erenkervan/xgboost-classification

In [1]:
import matplotlib.pyplot as plt
# import seaborn as sns
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score, roc_auc_score,roc_curve
from sklearn.preprocessing import StandardScaler, LabelEncoder
from xgboost import XGBClassifier
import time
import numpy as np 
import pandas as pd
import os
from sklearn.preprocessing import LabelBinarizer
import torch
import torch.nn.functional as F
import torch.nn as nn




In [2]:
df1=pd.read_csv('Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv')

In [3]:
df2=pd.read_csv('Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv')

In [4]:
df3=pd.read_csv('Friday-WorkingHours-Morning.pcap_ISCX.csv')

In [5]:
df4=pd.read_csv('Monday-WorkingHours.pcap_ISCX.csv')

In [6]:
df5=pd.read_csv('Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv')

In [7]:
df6=pd.read_csv('Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv')

In [8]:
df7=pd.read_csv('Tuesday-WorkingHours.pcap_ISCX.csv')

In [9]:
df8=pd.read_csv('Wednesday-workingHours.pcap_ISCX.csv')

In [10]:
# og=pd.read_csv('%Y-%m-%d-%H_%M_%S_ISCX.csv')
og=pd.read_csv('/home/vijay/Rutgers/course_work/553/project/TCPDUMP_and_CICFlowMeter/csv/dump_ISCX.csv')

In [11]:
og.drop(['Flow ID','Src IP','Src Port','Dst IP','Protocol','Timestamp'], inplace=True, axis=1)
og.rename(columns = {'Dst Port':' Destination Port', 'Total Bwd packets':'Total Backward Packets','Packet Length Min':' Min Packet Length','Packet Length Max':' Max Packet Length','CWR Flag Count':' CWE Flag Count','Fwd Segment Size Avg':' Avg Fwd Segment Size','Bwd Segment Size Avg':' Avg Bwd Segment Size','Fwd Bytes/Bulk Avg':'Fwd Avg Bytes/Bulk'}, inplace = True)
og.rename(columns={'Fwd Packet/Bulk Avg':' Fwd Avg Packets/Bulk','Fwd Bulk Rate Avg':' Fwd Avg Bulk Rate','Bwd Bytes/Bulk Avg':' Bwd Avg Bytes/Bulk','Bwd Packet/Bulk Avg':' Bwd Avg Bytes/Bulk','Bwd Bulk Rate Avg':'Bwd Avg Bulk Rate','FWD Init Win Bytes':'Init_Win_bytes_forward','Bwd Init Win Bytes':' Init_Win_bytes_backward','Fwd Act Data Pkts':' act_data_pkt_fwd'}, inplace=True)
og.rename(columns={'Fwd Seg Size Min':' min_seg_size_forward'}, inplace=True)

In [12]:

df = pd.concat([df1,df2])
df = pd.concat([df,df3])
df = pd.concat([df,df4])
df = pd.concat([df,df5])
df = pd.concat([df,df6])
df = pd.concat([df,df7])
df = pd.concat([df,df8])
df.drop(' Fwd Header Length', inplace=True, axis=1)
df.shape

(2830743, 78)

In [13]:
# drop duplicate rows
df =  df.drop_duplicates(keep="first")
df.reset_index(drop=True,inplace=True)

og =  og.drop_duplicates(keep="first")
og.reset_index(drop=True,inplace=True)

In [14]:
for i in df.columns:
    df = df[df[i] != "Infinity"]
    df = df[df[i] != np.nan]
    df = df[df[i] != ",,"]

In [15]:
old_memory_usage = df.memory_usage().sum()

In [16]:
og_old_memory_usage = og.memory_usage().sum()

In [17]:
new_memory_usage = df.memory_usage().sum()
old_vs_new = (old_memory_usage - new_memory_usage) / old_memory_usage * 100
print(f"%{old_vs_new} lower memory usage")

%0.0 lower memory usage


In [18]:
og_new_memory_usage = og.memory_usage().sum()
old1_vs_new1 = (og_old_memory_usage - og_new_memory_usage) / og_old_memory_usage * 100
print(f"%{old1_vs_new1} lower memory usage")

%0.0 lower memory usage


In [19]:
df.columns =  df.columns.str.strip()
og.columns =  og.columns.str.strip()

In [20]:
# drop nan and infinite rows
df = df[~df.isin([np.nan, np.inf, -np.inf]).any(1)]
og = og[~og.isin([np.nan, np.inf, -np.inf]).any(1)]

TypeError: NDFrame._add_numeric_operations.<locals>.any() takes 1 positional argument but 2 were given

In [22]:
# merging similar classes with low instances
df["Label"] = df["Label"].replace(["Web Attack � Brute Force","Web Attack � XSS","Web Attack � Sql Injection"],"Web Attack")


In [23]:
# #feature reduction 
# #dropping very high correlated features 
# corr_matrix = df.corr().abs()

# # Select upper triangle of correlation matrix
# upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# # Find features with correlation greater than 0.95
# to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
# print(to_drop)
# # Drop features 
# df =  df.drop(to_drop, axis=1)
# og =  og.drop(to_drop, axis=1)
# df.shape

In [24]:
l= np.array(['BENIGN', 'DDoS', 'PortScan', 'Bot', 'Infiltration', 'Web Attack',
       'FTP-Patator', 'SSH-Patator', 'DoS slowloris', 'DoS Slowhttptest',
       'DoS Hulk', 'DoS GoldenEye', 'Heartbleed'])
r= [0,1,2,3,4,5,6,7,8,9,10,11,12]
# y_dense = LabelBinarizer().fit_transform(l)


In [25]:
# x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3,random_state=42,stratify=y)
x_train=df.drop(["Label"],axis=1)
x_test=og.drop(["Label"],axis=1)

y_train=df["Label"]
y_test =og["Label"]
y_train=y_train.replace(l,r)
y_test=y_test.replace(l,r)


In [26]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Fwd Header Length
- Total Fwd Packet
- Total Length of Bwd Packet
- Total Length of Fwd Packet
Feature names seen at fit time, yet now missing:
- Bwd Avg Packets/Bulk
- Fwd Header Length.1
- Total Fwd Packets
- Total Length of Bwd Packets
- Total Length of Fwd Packets

  warnings.warn(message, FutureWarning)


In [27]:
x_test

array([[ 1.80717282e+00, -3.28961620e-01, -1.16836724e-02, ...,
         2.44144230e+08,  6.55802889e+07, -1.58303604e-01],
       [ 1.80748839e+00, -3.28960059e-01, -1.16836724e-02, ...,
         2.44144230e+08,  6.55802889e+07, -1.58301692e-01],
       [-4.33791542e-01, -4.70903028e-01, -1.04248923e-02, ...,
        -1.16080147e-01,  6.55802889e+07,  6.83442797e+07],
       ...,
       [-4.54304043e-01, -4.70834257e-01, -1.16836724e-02, ...,
        -1.16080147e-01,  6.55802889e+07,  6.83442797e+07],
       [-4.54304043e-01, -4.70834172e-01, -1.16836724e-02, ...,
        -1.16080147e-01,  6.55802889e+07,  6.83442797e+07],
       [ 1.80790916e+00, -4.70912934e-01, -1.16836724e-02, ...,
        -1.16080147e-01,  6.55802889e+07,  6.83442797e+07]])

In [28]:
x_train

array([[ 2.42859638, -0.4709143 , -0.01042489, ..., -0.11608015,
        -0.38113744, -0.36176379],
       [ 2.43853706, -0.47091129, -0.01168367, ..., -0.11608015,
        -0.38113744, -0.36176379],
       [ 2.43858965, -0.47091291, -0.01168367, ..., -0.11608015,
        -0.38113744, -0.36176379],
       ...,
       [ 2.59506321, -0.47091205, -0.01042489, ..., -0.11608015,
        -0.38113744, -0.36176379],
       [-0.45430404, -0.44115131, -0.00538977, ..., -0.11608015,
        -0.38113744, -0.36176379],
       [-0.45430404, -0.46821976, -0.00790733, ..., -0.11608015,
        -0.38113744, -0.36176379]])

In [29]:
# class MnistModel(nn.Module):
#     def __init__(self):
#         super().__init__()
#         #layers of the model
#         self.l1 = nn.Linear(input_size, 512)
#         self.l2 = nn.Linear(512, 10)
#         self.flat = nn.Flatten()
 
#     def forward(self, x):
#         #one forward pass
#         out = self.flat(x)
#         out = F.relu(self.l1(out))
#         out = self.l2(out)
#         return out

In [30]:
# start = time.time()
xgb = XGBClassifier(random_state=42)

xgb.fit(x_train,y_train)
xgbpreds = xgb.predict(x_test)
# print("Time", time.time()-start)
# print("Accuracy",accuracy_score(y_test,xgbpreds))
# print(classification_report(y_test,xgbpreds))

In [31]:
print(np.unique(xgbpreds))
xgbpreds=list(xgbpreds)

[0]


In [32]:
temp=pd.DataFrame(xgbpreds,columns=['Label_prediction'])
temp=temp.replace(r,l)
temp

,Label_prediction
0,BENIGN
1,BENIGN
2,BENIGN
3,BENIGN
4,BENIGN
5,BENIGN
6,BENIGN
7,BENIGN
8,BENIGN
9,BENIGN


In [33]:
temp.to_csv('label_pred.csv')

In [34]:
# save in JSON format
xgb.save_model("dis_model.json")
# save in text format
xgb.save_model("dis_model.txt")